In [69]:
import pandas as pd

# Práca
bridges_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\df_bridges_2024_all.csv"

# Load the CSV data into a DataFrame
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')

In [70]:
# batch = filtered_bridges[filtered_bridges["Číslo_PK"] == "2753"]
# for _, item in batch.iterrows():
#     print(f'{item["ID_mosta"]};{item["Zemepisná_šírka"]}, {
#           item["Zemepisná_dĺžka"]}; {item["Dĺžka_premostenia"]}')

In [71]:
# Assuming df_bridges_all is your DataFrame
columns_to_convert = ['Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška', 'Normálna_zaťažiteľnosť',
                      'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t', 'Počet_otvorov',
                      'Dĺžka_nosnej_konštrukcie_m', 'Dĺžka_premostenia', 'Volná_šírka_mosta',
                      'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2', 'Plocha_mosta_m2']

# Convert specified columns to float
df_bridges_all[columns_to_convert] = df_bridges_all[columns_to_convert].apply(
    pd.to_numeric, errors='coerce')

# Convert Rok_postavenia to integer, assign NaN if not possible
df_bridges_all['Rok_postavenia'] = pd.to_numeric(
    df_bridges_all['Rok_postavenia'], errors='coerce')

In [72]:
# Filter bridges in the Nitra region with Trieda_PK as "cesta I. triedy" or "cesta II. triedy"
filtered_bridges = df_bridges_all[

    (df_bridges_all["Správca_úseku"] == 'BBRSC-STRED') |
    (df_bridges_all["Správca_úseku"] == 'BBRSC-SEVER') |
    (df_bridges_all["Správca_úseku"] == 'BBRSC-JUH')
]

# Reindex the DataFrame
filtered_bridges.reset_index(drop=True, inplace=True)

filtered_bridges = filtered_bridges[filtered_bridges["n_2023"] != 0]

filtered_bridges = filtered_bridges[filtered_bridges["Material"]
                                    == "prefabrikovaný predpätý betón"]

filtered_bridges = filtered_bridges[
    (filtered_bridges["n_2024"] == 4) |
    (filtered_bridges["n_2024"] == 5)
]

# Replace double space with single space in each item of the "Predmet_premostenia" column
filtered_bridges["Predmet_premostenia"] = filtered_bridges["Predmet_premostenia"].str.replace(
    "  ", " ")


# Display the filtered DataFrame
filtered_bridges.shape

(111, 44)

In [73]:
import pandas as pd

path = r"C:\Users\relia\Documents\GitHub\Bridges\06_SC_BB\IoT\Mosty.xlsx"

# Load the Excel file into a DataFrame
df = pd.read_excel(path)

# Display the first few rows to ensure it loaded correctly
print(df.head())

  ID_mosta                 GPS  Dlzka  Intenzita   PK
0    M4403  48.07523, 19.17479  14.60       1472  527
1    M2559  48.07037, 19.25055  12.00       1383  527
2     M403  48.07406, 19.26022  15.30       1383  527
3    M6025  48.16523, 19.35814  17.00       3860  527
4     M381   48.1684, 19.36058  15.78       3860  527


In [74]:
df.columns

Index(['ID_mosta', 'GPS', 'Dlzka', 'Intenzita', 'PK'], dtype='object')

In [77]:
# Path to the GPX file
gpx_file = r"C:\Users\relia\Documents\GitHub\Bridges\06_SC_BB\IoT\Routes\M7230_alt.gpx"

# Read the GPX file as text
with open(gpx_file, "r", encoding="utf-8") as file:
    gpx_content = file.read()

# Extract <rtept> sections
# Split by <rtept> and ignore the first part (metadata)
rtept_sections = gpx_content.split("<rtept")[1:]

# Initialize totals
total_distance = 0.0
total_time = 0.0

# Process each <rtept>
for rtept in rtept_sections:
    # Extract <gh:distance> value
    if "<gh:distance>" in rtept:
        start_distance = rtept.find("<gh:distance>") + len("<gh:distance>")
        end_distance = rtept.find("</gh:distance>")
        distance = float(rtept[start_distance:end_distance])
        total_distance += distance

    # Extract <gh:time> value
    if "<gh:time>" in rtept:
        start_time = rtept.find("<gh:time>") + len("<gh:time>")
        end_time = rtept.find("</gh:time>")
        time = float(rtept[start_time:end_time])
        total_time += time

# Convert totals to kilometers and minutes
total_distance_km = total_distance / 1000  # Convert meters to kilometers
# Convert milliseconds to minutes
total_time_minutes = total_time / (1000 * 60)


# Print results
print(f"Total Distance: {total_distance_km:.2f} kilometers")
print(f"Total Time: {total_time_minutes:.2f} minutes")

Total Distance: 13.87 kilometers
Total Time: 14.53 minutes


In [78]:
import os

# Path to the folder with GPX files
folder_path = r"C:\Users\relia\Documents\GitHub\Bridges\06_SC_BB\IoT\Routes"

# Function to extract distance and time from a GPX file


def extract_distance_time(gpx_file):
    with open(gpx_file, "r", encoding="utf-8") as file:
        gpx_content = file.read()

    # Initialize totals
    total_distance = 0.0
    total_time = 0.0

    # Extract <rtept> sections
    rtept_sections = gpx_content.split("<rtept")[1:]  # Split by <rtept>

    # Process each <rtept>
    for rtept in rtept_sections:
        if "<gh:distance>" in rtept:
            start_distance = rtept.find("<gh:distance>") + len("<gh:distance>")
            end_distance = rtept.find("</gh:distance>")
            distance = float(rtept[start_distance:end_distance])
            total_distance += distance
        if "<gh:time>" in rtept:
            start_time = rtept.find("<gh:time>") + len("<gh:time>")
            end_time = rtept.find("</gh:time>")
            time = float(rtept[start_time:end_time])
            total_time += time

    # Convert to kilometers and minutes
    total_distance_km = total_distance / 1000  # meters to kilometers
    total_time_minutes = total_time / (1000 * 60)  # milliseconds to minutes
    return total_distance_km, total_time_minutes


# Iterate over each row in the DataFrame
clear_distances = []
clear_times = []
alt_distances = []
alt_times = []

for _, row in df.iterrows():
    bridge_id = row["ID_mosta"]
    clear_file = os.path.join(folder_path, f"{bridge_id}_clear.gpx")
    alt_file = os.path.join(folder_path, f"{bridge_id}_alt.gpx")

    # Check if files exist
    if os.path.exists(clear_file):
        clear_distance, clear_time = extract_distance_time(clear_file)
    else:
        clear_distance, clear_time = None, None

    if os.path.exists(alt_file):
        alt_distance, alt_time = extract_distance_time(alt_file)
    else:
        alt_distance, alt_time = None, None

    # Append results
    clear_distances.append(clear_distance)
    clear_times.append(clear_time)
    alt_distances.append(alt_distance)
    alt_times.append(alt_time)

# Add new columns to the DataFrame
df["Clear_Distance"] = clear_distances
df["Clear_Time"] = clear_times
df["Alt_Distance"] = alt_distances
df["Alt_Time"] = alt_times

In [82]:
df["Diff_Time"] = df["Intenzita"] * (df["Alt_Time"] - df["Clear_Time"])
df["Diff_Dist"] = df["Intenzita"] * (df["Alt_Distance"] - df["Clear_Distance"])
df["Price"] = (df["Diff_Dist"] / 100) * 6 * 1.516

In [85]:
df["Price"].describe()

count      55.000000
mean     1160.902612
std      1286.486635
min         3.838255
25%       141.400360
50%       901.444377
75%      1818.799462
max      6819.611726
Name: Price, dtype: float64

In [1]:
# # Sort the DataFrame by the "Price" column in descending order
# df_sorted = df.sort_values(by="Price", ascending=False)

# df_sorted

df

NameError: name 'df' is not defined

In [ ]:
import matplotlib.pyplot as plt

# Define the bridge ID to find
bridge_to_find = "M4403"

# Locate the bridge in the DataFrame
bridge_row = filtered_bridges[filtered_bridges["ID_mosta"] == bridge_to_find]

bridge_row.iloc[0][["ID_mosta", "Číslo_PK", "správcovské_číslo", "Názov_mosta",
                    "Rok_postavenia", "Predmet_premostenia", "Dĺžka_premostenia", "Okres", "n_2024"]]

ID_mosta                                                           M4403
Číslo_PK                                                             527
správcovské_číslo                                                      7
Názov_mosta            000527-007 Most cez Veľký potok v obci Kosihy ...
Rok_postavenia                                                      1963
Predmet_premostenia                                            Vodný tok
Dĺžka_premostenia                                                   14.6
Okres                                                        Veľký Krtíš
n_2024                                                                 5
Name: 74, dtype: object

In [112]:
bridge_row.iloc[0]["Názov_mosta"]

'000527-007 Most cez Veľký potok v obci Kosihy nad Ipľom'

In [111]:
import xml.etree.ElementTree as ET
import folium

import os

folder_path = r"C:\Users\relia\Documents\GitHub\Bridges\06_SC_BB\IoT\Routes"

# List all files in the folder
files = os.listdir(folder_path)

# Define the bridge ID
bridge_id = bridge_to_find

# Filter the files for the specific bridge
clear_route = None
alternative_route = None

for file in files:
    if file.startswith(bridge_id):
        if "_clear" in file:
            clear_route = os.path.join(folder_path, file)
        elif "_alt" in file:
            alternative_route = os.path.join(folder_path, file)

# Assign variables for clear and alternative routes
print(f"Clear route for {bridge_id}: {clear_route}")
print(f"Alternative route for {bridge_id}: {alternative_route}")


def extract_coordinates(gpx_file):
    tree = ET.parse(gpx_file)
    root = tree.getroot()
    namespace = {"default": "http://www.topografix.com/GPX/1/1"}
    coordinates = []
    for trkpt in root.findall(".//default:trkpt", namespace):
        lat = float(trkpt.attrib['lat'])
        lon = float(trkpt.attrib['lon'])
        coordinates.append((lat, lon))
    return coordinates


# Get bridge latitude and longitude
bridge_lat = bridge_row.iloc[0]["Zemepisná_šírka"]
bridge_lon = bridge_row.iloc[0]["Zemepisná_dĺžka"]
bridge_name = bridge_row.iloc[0]["Názov_mosta"]

# Extract coordinates for clear and alternative routes
clear_coordinates = extract_coordinates(clear_route)
alt_coordinates = extract_coordinates(alternative_route)

# Create a Folium map centered on the first point of the clear route
route_map = folium.Map(location=clear_coordinates[0], zoom_start=14)

# Add a marker for the bridge
folium.Marker(
    location=[bridge_lat, bridge_lon],
    tooltip="Bridge Location",
    icon=folium.Icon(color="blue", icon="info-sign")
).add_to(route_map)

# Add the clear route to the map
folium.PolyLine(clear_coordinates, color="green", weight=5,
                opacity=0.8, tooltip="Clear Route").add_to(route_map)

# Add the alternative route to the map
folium.PolyLine(alt_coordinates, color="red", weight=5,
                opacity=0.8, tooltip="Alternative Route").add_to(route_map)

route_map.save(f"most.html")

Clear route for M4403: C:\Users\relia\Documents\GitHub\Bridges\06_SC_BB\IoT\Routes\M4403_clear.gpx
Alternative route for M4403: C:\Users\relia\Documents\GitHub\Bridges\06_SC_BB\IoT\Routes\M4403_alt.gpx
